### Этот код выполняет классификацию электронных писем на спам и не спам, используя алгоритм Locality Sensitive Hashing (LSH) с помощью MinHash и MinHashLSH из библиотеки datasketch. Каждое письмо представлено как множество уникальных слов (stems), и на основе этого множества строится MinHash. Далее производится поиск совпадений MinHash с помощью LSH для определения, является ли письмо спамом или не спамом. После классификации вычисляются метрики качества, такие как матрица ошибок, процент ошибок и точность классификации.

In [1]:
# Импорт необходимых модулей
import os  # Модуль для работы с операционной системой
import pickle  # Модуль для сериализации и десериализации объектов Python
import email_read_util  # Модуль для чтения и обработки электронных писем
from datasketch import MinHash, MinHashLSH  # Модули для работы с MinHash и MinHashLSH

In [2]:
# Определение директории с данными и файла с метками
DATA_DIR = 'datasets/trec07p/data/'  # Путь к директории с данными
LABELS_FILE = 'datasets/trec07p/full/index'  # Путь к файлу с метками
TRAINING_SET_RATIO = 0.7  # Пропорция разделения набора данных на тренировочный и тестовый
labels = {}  # Словарь для хранения меток (имя файла: метка)


In [4]:
# Чтение и обработка файла меток
with open(LABELS_FILE) as f:
    for line in f:
        line = line.strip()
        label, key = line.split()
        labels[key.split('/')[-1]] = 1 if label.lower() == 'ham' else 0

In [5]:
# Разделение корпуса на тренировочный и тестовый наборы
filelist = os.listdir(DATA_DIR)
X_train = filelist[:int(len(filelist)*TRAINING_SET_RATIO)]
X_test = filelist[int(len(filelist)*TRAINING_SET_RATIO):]

In [6]:
# Извлечение только файлов со спамом для вставки в сопоставитель LSH
spam_files = [x for x in X_train if labels[x] == 0]

In [7]:
# Инициализация сопоставителя MinHashLSH с порогом Жаккара 0.5 и 128 функциями перестановки MinHash
lsh = MinHashLSH(threshold=0.5, num_perm=128)

In [8]:
# Заполнение сопоставителя LSH тренировочными MinHashes для спама
for idx, f in enumerate(spam_files):
    minhash = MinHash(num_perm=128)
    stems = email_read_util.load(os.path.join(DATA_DIR, f))
    if len(stems) < 2: continue
    for s in stems:
        minhash.update(s.encode('utf-8'))
    lsh.insert(f, minhash)

In [9]:
def lsh_predict_label(stems):
    '''
    Queries the LSH matcher and returns:
        0 if predicted spam
        1 if predicted ham
       -1 if parsing error
    '''
    minhash = MinHash(num_perm=128)
    if len(stems) < 2:
        return -1
    for s in stems:
        minhash.update(s.encode('utf-8'))
    matches = lsh.query(minhash)
    if matches:
        return 0
    else:
        return 1

In [10]:
# Инициализация переменных для подсчета статистики классификации
fp = 0  # Ложноположительные (ошибочно определенные как спам)
tp = 0  # Истинноположительные (правильно определенные как спам)
fn = 0  # Ложноотрицательные (ошибочно определенные как не спам)
tn = 0  # Истинноотрицательные (правильно определенные как не спам)

# Проверка каждого письма из тестового набора и классификация с помощью сопоставителя LSH
for filename in X_test:
    path = os.path.join(DATA_DIR, filename)
    if filename in labels:
        label = labels[filename]
        stems = email_read_util.load(path)
        if not stems:
            continue
        pred = lsh_predict_label(stems)
        if pred == -1:
            continue
        elif pred == 0:
            if label == 1:
                fp = fp + 1
            else:
                tp = tp + 1
        elif pred == 1:
            if label == 1:
                tn = tn + 1
            else:
                fn = fn + 1

In [11]:
# Вывод матрицы ошибок в HTML-таблице
from IPython.display import HTML, display
conf_matrix = [[tn, fp],
               [fn, tp]]
display(HTML('<table><tr>{}</tr></table>'.format(
    '</tr><tr>'.join('<td>{}</td>'.format(
        '</td><td>'.join(str(_) for _ in row)) 
                     for row in conf_matrix))))

7350,136
2241,11038


In [12]:
# Вывод процентной матрицы ошибок в HTML-таблице
count = tn + tp + fn + fp
percent_matrix = [["{:.1%}".format(tn/count), "{:.1%}".format(fp/count)],
                  ["{:.1%}".format(fn/count), "{:.1%}".format(tp/count)]]
display(HTML('<table><tr>{}</tr></table>'.format(
    '</tr><tr>'.join('<td>{}</td>'.format(
        '</td><td>'.join(str(_) for _ in row)) 
                     for row in percent_matrix))))

35.4%,0.7%
10.8%,53.2%


In [13]:
# Вывод процента правильно классифицированных писем
print("Classification accuracy: {}".format("{:.1%}".format((tp+tn)/count)))

Classification accuracy: 88.6%
